In [ ]:
#get new csv from nfl_cleaning.pynb
df = pd.read_csv('new_csv')

In [ ]:
df = df.sort_values('Date')

In [ ]:
#shift by 1 to align the data so that the target is representative of the next game
#this is reflected in target column

#https://www.youtube.com/watch?v=egTylm6C2is

def target_column(group, shift_steps=-1, column="Team1Won", target_name="target"):
    group[target_name] = group[column].shift(shift_steps)
    return group



df = df.groupby("team1", group_keys=False).apply(target_column)

In [ ]:
#check 
df[df["team1"] == "SFO"]

In [ ]:
df["target"].value_counts()

In [ ]:
# use the sequentialfeatureselector? or try something else? PCA wont work here

In [ ]:
#rolling averages of the past. 

def add_rolling_averages():


In [ ]:
import pandas as pd

In [ ]:
#get new csv from nfl_cleaning.pynb
df = pd.read_csv('new_csv')

In [ ]:
df = df.sort_values('Date')

In [ ]:
# new indexes
df.index= range(len(df))

In [ ]:
#shift by 1 to align the data so that the target is representative of the next game
#this is reflected in target column

#https://www.youtube.com/watch?v=egTylm6C2is

def target_column(group, shift_steps=-1, column="Team1Won", target_name="target"):
    group[target_name] = group[column].shift(shift_steps)
    return group

df = df.groupby("team1", group_keys=False).apply(target_column)

In [ ]:
#check 
df[df["team1"] == "SFO"]

In [ ]:

df["Team1Won"].value_counts()

In [ ]:
df["target"].value_counts()

In [ ]:
# use the sequentialfeatureselector? or try something else? PCA wont work here

In [ ]:
#rolling averages of the past. 

def add_rolling_averages():


df = add_rolling_averages(df, columns=columns_to_average, window=10)